In [53]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px 
import plotly.graph_objects as go 
from sklearn.model_selection import TimeSeriesSplit
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score
from colorama import Fore, Style
from IPython.display import display, HTML 
print('Setup Complete')

Setup Complete


In [54]:
data = pd.read_csv('Apple_News_Analyzed.csv')

data.head(5)

,ticker,Date,title,content,Open,High,Low,Close,Adj Close,Volume,...,clean_title,clean_content,title_tokenize,content_tokenize,stopless_title,stopless_content,title_stemmed,content_stemmed,sentiment_title,sentiment_content
0,AAPL,7/23/2012,Summer Heat Scorches Europe And U S,Europe flares as summer heat continues Summer...,21.228571,21.639286,20.989643,21.565357,18.257221,487975600.0,...,summer heat scorches europe and u s,europe flares as summer heat continues summer...,['summer heat scorches europe and u s '],['europe flares as summer heat continues summ...,['summer heat scorches europe and u s '],['europe flares as summer heat continues summ...,['summer heat scorches europe and u s '],['europe flares as summer heat continues summ...,0,1
1,AAPL,7/23/2012,Apple Earnings Preview Quarterly Dip On Deck,Last quarter Apple AAPL reported the 2nd bes...,21.692142,21.774286,21.375357,21.461430,18.169237,565132400.0,...,apple earnings preview quarterly dip on deck,last quarter apple aapl reported the 2nd bes...,['apple earnings preview quarterly dip on dec...,['last quarter apple aapl reported the 2nd b...,['apple earnings preview quarterly dip on dec...,['last quarter apple aapl reported the 2nd b...,['apple earnings preview quarterly dip on dec...,['last quarter apple aapl reported the 2nd b...,0,1
2,AAPL,7/23/2012,Trade Apple After Earnings,It may look like a spider web but the mishmas...,20.516430,20.742857,20.357143,20.534643,17.384607,877312800.0,...,trade apple after earnings,it may look like a spider web but the mishmas...,['trade apple after earnings'],['it may look like a spider web but the mishm...,['trade apple after earnings'],['it may look like a spider web but the mishm...,['trade apple after earn'],['it may look like a spider web but the mishm...,0,1
3,AAPL,7/24/2012,Market Bait And Switch,That is the sound we are going to hear soon fr...,20.705713,20.728571,20.370001,20.531429,17.381897,406632800.0,...,market bait and switch,that is the sound we are going to hear soon fr...,['market bait and switch'],['that is the sound we are going to hear soon ...,['market bait and switch'],['that is the sound we are going to hear soon ...,['market bait and switch'],['that is the sound we are going to hear soon ...,0,-1
4,AAPL,7/27/2012,Will AAPL Fall From The Tree,Apple s AAPL sales for the third quarter mis...,20.536072,20.922501,20.413929,20.898571,17.692715,403936400.0,...,will aapl fall from the tree,apple s aapl sales for the third quarter mis...,['will aapl fall from the tree '],['apple s aapl sales for the third quarter m...,['will aapl fall from the tree '],['apple s aapl sales for the third quarter m...,['will aapl fall from the tree '],['apple s aapl sales for the third quarter m...,0,1


In [55]:
data = data.dropna()
m, n = data.shape

train = data[:1400]
test = data[1400:m]

print('Train shape: ', train.shape)
print('Test shape: ', test.shape)

Train shape:  (1400, 21)
Test shape:  (393, 21)


In [56]:
tss = TimeSeriesSplit(n_splits=15)

predictors = ["Open","High","Low","Close","sentiment_title","sentiment_content",]

def model(model, predictors):
    for train_index, test_index in tss.split(data):
        train = data.iloc[train_index]
        test = data.iloc[test_index]

        predictors = [
            "Open",
            "High",
            "Low",
            "Close",
            "sentiment_title",
            "sentiment_content",
        ]

        model.fit(train[predictors], train["Close"])

In [57]:
def pred(model, predictors):
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index)
    
    return preds

In [58]:
def eval(y_true, y_pred, tolerance=0.05):
    within_tolerance = np.abs((y_true - y_pred) / y_true) <= tolerance
    return np.mean(within_tolerance)

In [59]:
rfr = RandomForestRegressor(n_estimators=100, min_samples_split=50, random_state=1)
model(rfr, predictors)
rfr_preds = pred(rfr, predictors)

rfr_mse = mean_squared_error(test["Close"], rfr_preds)
print("MSE in RandomForestRegressor predictions: ", rfr_mse)
rfraccuracy = eval(test["Close"], rfr_preds, tolerance=0.005)
print("Accuracy within 0.5% tolerance:", rfraccuracy)

MSE in RandomForestRegressor predictions:  1.129874413432808
Accuracy within 0.5% tolerance: 0.3333333333333333


In [60]:
cat = CatBoostRegressor()
model(cat, predictors)
cat_preds = pred(cat, predictors)

cat_mse = mean_squared_error(test["Close"], cat_preds)
print("MSE in CatBoostRegressor predictions: ", cat_mse)
cataccuracy = eval(test["Close"], cat_preds, tolerance=0.005)
print("Accuracy within 0.5% tolerance:", cataccuracy)

Learning rate set to 0.029011
0:	learn: 1.9562741	total: 1.17ms	remaining: 1.17s
1:	learn: 1.9156107	total: 2.06ms	remaining: 1.03s
2:	learn: 1.8755677	total: 3.18ms	remaining: 1.06s
3:	learn: 1.8408326	total: 3.74ms	remaining: 932ms
4:	learn: 1.8049348	total: 4.27ms	remaining: 850ms
5:	learn: 1.7667982	total: 5ms	remaining: 828ms
6:	learn: 1.7309513	total: 5.75ms	remaining: 816ms
7:	learn: 1.6967095	total: 6.51ms	remaining: 808ms
8:	learn: 1.6590899	total: 7.25ms	remaining: 799ms
9:	learn: 1.6218707	total: 7.99ms	remaining: 791ms
10:	learn: 1.5874474	total: 8.77ms	remaining: 788ms
11:	learn: 1.5533800	total: 9.52ms	remaining: 784ms
12:	learn: 1.5211805	total: 10.3ms	remaining: 784ms
13:	learn: 1.4924867	total: 11.1ms	remaining: 780ms
14:	learn: 1.4584295	total: 11.8ms	remaining: 774ms
15:	learn: 1.4293308	total: 12.3ms	remaining: 757ms
16:	learn: 1.3978385	total: 12.9ms	remaining: 747ms
17:	learn: 1.3704290	total: 13.5ms	remaining: 739ms
18:	learn: 1.3427473	total: 14.2ms	remaining: 7

In [61]:
xgb = XGBRegressor()
model(xgb, predictors)
xgb_preds = pred(xgb, predictors)

xgb_mse = mean_squared_error(test['Close'], xgb_preds)
print('MSE in XGBRegressor predictions:\n', xgb_preds)
xgbaccuracy = eval(test["Close"], xgb_preds, tolerance=0.005)
print("Accuracy within 0.5% tolerance:", xgbaccuracy)

MSE in XGBRegressor predictions:
 1400    41.824150
1401    43.276798
1402    43.109501
1403    42.978134
1404    42.762844
          ...    
1788    52.284416
1789    51.365776
1790    52.231167
1791    53.401321
1792    53.507690
Length: 393, dtype: float32
Accuracy within 0.5% tolerance: 0.8295165394402035


In [62]:
lgbm = LGBMRegressor()
model(lgbm, predictors)
lgbm_preds = pred(lgbm, predictors)

lgbm_mse = mean_squared_error(test["Close"], lgbm_preds)
print("MSE in LGBMRegressor predictions: ", lgbm_mse)
lgbmaccuracy = eval(test["Close"], lgbm_preds, tolerance=0.005)
print("Accuracy within 0.5% tolerance:", lgbmaccuracy)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000039 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 159
[LightGBM] [Info] Number of data points in the train set: 113, number of used features: 5
[LightGBM] [Info] Start training from score 21.741419
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [63]:
trace1 = go.Scatter(x=train.index, y=train["Close"], mode="lines", name="Train")
trace2 = go.Scatter(x=test.index, y=test["Close"], mode="lines", name="Val")
trace3 = go.Scatter(x=test.index, y=rfr_preds, mode="lines", name="RFR Preds")
trace4 = go.Scatter(x=test.index, y=cat_preds, mode="lines", name="CatBoost Preds")
trace5 = go.Scatter(x=test.index, y=xgb_preds, mode="lines", name="XGBoost Preds")
trace6 = go.Scatter(x=test.index, y=lgbm_preds, mode="lines", name="LGMB Preds")

# Combine traces
fig = go.Figure([trace1, trace2, trace3, trace4, trace5, trace6])

# Update layout
fig.update_layout(
    title="Model Predictions",
    xaxis=dict(title="Date", tickfont=dict(size=14)),
    yaxis=dict(title="Close Price USD ($)", tickfont=dict(size=14)),
    legend=dict(x=0.02, y=0.98),
    plot_bgcolor="black",
    paper_bgcolor="black",
    font=dict(color="white"),
)

# Show plot
fig.show()

In [65]:
ensemble = VotingRegressor(estimators=[('rfr', rfr),
                                        ('cat', cat), 
                                        ('xgb', xgb), 
                                        ('lgbm', lgbm)])
model(ensemble, predictors)

Learning rate set to 0.029011
0:	learn: 1.9562741	total: 1.3ms	remaining: 1.3s
1:	learn: 1.9156107	total: 2.32ms	remaining: 1.16s
2:	learn: 1.8755677	total: 3.29ms	remaining: 1.09s
3:	learn: 1.8408326	total: 3.76ms	remaining: 937ms
4:	learn: 1.8049348	total: 4.27ms	remaining: 850ms
5:	learn: 1.7667982	total: 5.28ms	remaining: 875ms
6:	learn: 1.7309513	total: 6.3ms	remaining: 893ms
7:	learn: 1.6967095	total: 7.42ms	remaining: 920ms
8:	learn: 1.6590899	total: 8.36ms	remaining: 920ms
9:	learn: 1.6218707	total: 9.58ms	remaining: 948ms
10:	learn: 1.5874474	total: 10.6ms	remaining: 949ms
11:	learn: 1.5533800	total: 11.8ms	remaining: 969ms
12:	learn: 1.5211805	total: 12.6ms	remaining: 957ms
13:	learn: 1.4924867	total: 13.4ms	remaining: 945ms
14:	learn: 1.4584295	total: 14.2ms	remaining: 932ms
15:	learn: 1.4293308	total: 14.8ms	remaining: 910ms
16:	learn: 1.3978385	total: 15.6ms	remaining: 904ms
17:	learn: 1.3704290	total: 16.6ms	remaining: 904ms
18:	learn: 1.3427473	total: 17.5ms	remaining: 9

In [66]:
ensemble_preds = pred(ensemble, predictors)
ens_mse = mean_squared_error(test["Close"], ensemble_preds)
print("MSE in LGBMRegressor predictions: ", ens_mse)
ensaccuracy = eval(test["Close"], ensemble_preds, tolerance=0.005)
print("Accuracy within 0.5% tolerance:", ensaccuracy)

MSE in LGBMRegressor predictions:  0.081199836778334
Accuracy within 0.5% tolerance: 0.7404580152671756


In [67]:
trace1 = go.Scatter(x=train.index, y=train["Close"], mode="lines", name="Train")
trace2 = go.Scatter(x=test.index, y=test["Close"], mode="lines", name="Val")
trace3 = go.Scatter(x=test.index, y=ensemble_preds, mode="lines", name="Ensemble")

# Combine traces
fig = go.Figure([trace1, trace2, trace3])

# Update layout
fig.update_layout(
    title="Ensemble Model Predictions",
    xaxis=dict(title="Date", tickfont=dict(size=14)),
    yaxis=dict(title="Close Price USD ($)", tickfont=dict(size=14)),
    legend=dict(x=0.02, y=0.98),
    plot_bgcolor="black",
    paper_bgcolor="black",
    font=dict(color="white"),
)

# Show plot
fig.show()